# 2020_10_15:Linear SVC

Linear SVC.Tuning the class weight and the "C" parameter.

# Imports

Class_weights -- C param -- training f1 score 

1:8 -- not passed -- 0.63 

1:4 -- not passed -- 0.67

1:4 -- 10 -- 0.67

1:4 -- 0.5 -- 0.67

1:2 -- not passed -- 0.66

1:3 --not passed -- 0.68

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt  
import wordcloud as wc 
import seaborn as sns 
import nltk
import re
from tqdm import tqdm
tqdm.pandas()

train_dset_df = pd.read_csv("./dataset/pp/2020_10_04_preprocessed_train.csv")
test_dset_df = pd.read_csv("./dataset/pp/2020_10_04_preprocessed_test.csv")
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(eval)
test_dset_df["preprocessed"] = test_dset_df["preprocessed"].progress_apply(eval)
train_dset_df = train_dset_df.drop(axis="columns", labels="Unnamed: 0")
test_dset_df = test_dset_df.drop(axis="columns", labels="Unnamed: 0")
from tqdm import tqdm 
tqdm.pandas()
import gc

100%|██████████| 522449/522449 [00:06<00:00, 81217.31it/s]


# Snowball Stemming

In [3]:
from nltk.stem import SnowballStemmer

In [4]:
snow_stem = SnowballStemmer("english")

In [5]:
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(lambda x: [snow_stem.stem(i) for i in x])

100%|██████████| 783673/783673 [00:53<00:00, 14725.23it/s]


In [6]:
test_dset_df["preprocessed"] = test_dset_df["preprocessed"].progress_apply(lambda x: [snow_stem.stem(i) for i in x])

100%|██████████| 522449/522449 [00:36<00:00, 14286.93it/s]


In [7]:
train_dset_df["preprocessed"] = train_dset_df["preprocessed"].progress_apply(lambda x: " ".join(x))

100%|██████████| 783673/783673 [00:00<00:00, 1115042.25it/s]


In [8]:
test_dset_df["preprocessed"] = test_dset_df["preprocessed"].progress_apply(lambda x: " ".join(x))

100%|██████████| 522449/522449 [00:00<00:00, 1107666.50it/s]


In [9]:
train_dset_questions = train_dset_df["preprocessed"]
test_dset_questions = test_dset_df["preprocessed"]

questions = pd.concat([train_dset_questions, test_dset_questions])

# TFIDFVectorizer

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [11]:
vectorizer = CountVectorizer()

In [12]:
vectorizer.fit(questions)

CountVectorizer()

In [54]:
sparse_x_train = vectorizer.transform(train_dset_df["preprocessed"])

In [55]:
sparse_x_train

<783673x84719 sparse matrix of type '<class 'numpy.int64'>'
	with 5499037 stored elements in Compressed Sparse Row format>

17

# Training

In [37]:
# from sklearn.kernel_approximation import Nystroem
from sklearn.svm import LinearSVC

In [38]:
svm = LinearSVC(verbose=2, max_iter = 4000, class_weight= {0:1, 1:3})

In [39]:
svm.fit(sparse_x_train, train_dset_df["target"])

[LibLinear]

LinearSVC(class_weight={0: 1, 1: 3}, max_iter=4000, verbose=2)

In [40]:
yhat = svm.predict(sparse_x_train)

In [41]:
train_dset_df["yhat"] = yhat

In [42]:
from sklearn.metrics import f1_score

In [43]:
wrongs = train_dset_df[train_dset_df["yhat"] != train_dset_df["target"]]

In [44]:
y = train_dset_df["target"]

In [45]:
f1_score(y, yhat)

0.6801767338663403

In [46]:
[(name, len(x)) for name, x in wrongs.groupby(by="target")]

[(0, 20552), (1, 12890)]

In [47]:
[(name, len(x)) for name, x in train_dset_df.groupby(by="target")]

[(0, 735222), (1, 48451)]

# Testing and writing output

In [48]:
sparse_x = vectorizer.transform(test_dset_df["preprocessed"])

In [49]:
yhat = svm.predict(sparse_x)

In [50]:
test_dset_df.question_text = yhat

In [51]:
test_dset_df = test_dset_df.drop(axis="columns", labels="preprocessed")
test_dset_df.head()

,qid,question_text
0,f56a9a31974dc66186e8,0
1,d957c3758060f45da303,0
2,ad822d5abaedb9e247b9,0
3,4e979c23eeb6a4bd1f2e,0
4,333cc031262566b8da49,0


In [52]:
test_dset_df = test_dset_df.rename(columns={"qid":"qid", "question_text":"target"})
test_dset_df.target = test_dset_df.target.apply(round)

In [53]:
test_dset_df.to_csv("./outputs/2020_10_15_b_testset_output.csv", index=False)